# Download einer Zeitung aus dem Zeitungsportal

In [1]:
%pip -q install pandas lxml ipywidgets tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
import requests as req
from lxml import etree
import ipywidgets as widgets
from tqdm import tqdm

# Basis-URL für die Deutsche Digitale Bibliothek API
ddb_api = "https://api.deutsche-digitale-bibliothek.de"

def download_newspaper_issues(zdb_id, output_folder):
    # Endpunkt für den Zugriff auf die Quelldaten einer Ressource
    ddb_api_source_call = "/items/{ddb_id}/source/record"
    
    # Endpunkt für die Suche nach Zeitungsausgaben basierend auf der ZDB-ID
    ddb_api_search_call = f"/search/index/newspaper-issues/select?q=zdb_id:{zdb_id} AND type:issue&fl=id,publication_date&wt=csv&rows=2147483647"
    
    # Namensräume für XML-Elemente im METS-Format
    mets_mods_namespaces = {
        "mets": "http://www.loc.gov/METS/",
        "xlink": "http://www.w3.org/1999/xlink"
    }
    
    # Lese die CSV-Daten mit Zeitungsinformationen von der Deutschen Digitalen Bibliothek
    df = pd.read_csv(req.utils.requote_uri(ddb_api + ddb_api_search_call))
    
    # Fortschrittsanzeige mit tqdm
    t = tqdm(df.iterrows(), total=len(df), desc="Download")
    
    # Iteriere über die Zeilen des DataFrames
    for index, row in t:
        # Extrahiere Informationen aus der aktuellen Zeile
        ddb_id = row['id']
        publication_date = row['publication_date']
        
        # Erstelle einen Pfad zum Speichern der heruntergeladenen Daten
        save_path = f"{output_folder}/{publication_date[0:10]}_{ddb_id}"
        
        # Erstelle den Ordner, falls er nicht existiert
        os.makedirs(save_path, exist_ok=True)
        
        # Baue die URL für den Zugriff auf die Quelldaten zusammen
        download_url = req.utils.requote_uri(ddb_api + ddb_api_source_call.format(ddb_id=ddb_id))
        
        # Führe die Anfrage für die Quelldaten durch
        download_resp = req.get(download_url, headers={"Accept": "application/xml"})
        
        # Parsen der XML-Daten mit lxml
        download_root = etree.fromstring(download_resp.content)
    
        # Extrahiere die URLs der Bildressourcen
        image_urls = download_root.xpath("//mets:mets/mets:fileSec/mets:fileGrp[@USE='MAX']/mets:file/mets:FLocat/@xlink:href", namespaces=mets_mods_namespaces)
    
        # Wenn die 'MAX'-fileGrp nicht existiert, verwende die 'DEFAULT'-fileGrp als Fallback
        if len(image_urls) < 1:
            image_urls = download_root.xpath("//mets:mets/mets:fileSec/mets:fileGrp[@USE='DEFAULT']/mets:file/mets:FLocat/@xlink:href", namespaces=mets_mods_namespaces)
        
        # Iteriere über die Bild-URLs und speichere die Dateien
        for i, image_url in enumerate(image_urls):
            image_filename = f"{save_path}/{str(i + 1).zfill(4)}.jpg"
            image_resp = req.get(image_url)
            
            with open(image_filename, 'wb') as image_file:
                image_file.write(image_resp.content)
    
            # Aktualisiere die Fortschrittsanzeige
            t.set_postfix_str(f"Bild {i + 1}/{len(image_urls)} für {publication_date[0:10]} gespeichert: {image_filename}")
    
        # Extrahiere die URLs der Volltextressourcen
        fulltext_urls = download_root.xpath("//mets:mets/mets:fileSec/mets:fileGrp[@USE='DDB_FULLTEXT']/mets:file/mets:FLocat/@xlink:href", namespaces=mets_mods_namespaces)
    
        # Wenn die 'DDB_FULLTEXT'-fileGrp nicht existiert, verwende die 'FULLTEXT'-fileGrp als Fallback
        if len(fulltext_urls) < 1:
            fulltext_urls = download_root.xpath("//mets:mets/mets:fileSec/mets:fileGrp[@USE='FULLTEXT']/mets:file/mets:FLocat/@xlink:href", namespaces=mets_mods_namespaces)
    
        
        # Iteriere über die Volltext-URLs und speichere die Dateien
        for i, fulltext_url in enumerate(fulltext_urls):
            fulltext_filename = f"{save_path}/{str(i + 1).zfill(4)}.xml"
            fulltext_resp = req.get(fulltext_url)
            
            with open(fulltext_filename, 'wb') as fulltext_file:
                fulltext_file.write(fulltext_resp.content)
    
            # Aktualisiere die Fortschrittsanzeige
            t.set_postfix_str(f"Volltext {i + 1}/{len(fulltext_urls)} für {publication_date[0:10]} gespeichert: {fulltext_filename}")


# Definiere den Endpunkt für die Suche nach Zeitungstitel
ddb_api_search_call = "/search/index/newspaper/select?q=hasLoadedIssues:true&fl=id,title&wt=csv&rows=2147483647"

# Lade die CSV-Datei von der Deutschen Digitalen Bibliothek API
df = pd.read_csv(req.utils.requote_uri(ddb_api + ddb_api_search_call))

# Erstelle eine neue Spalte 'label', die eine Kombination aus 'id' und 'title' ist
df['label'] = df.iloc[:, 0] + " | " + df.iloc[:, 1].apply(lambda x: x[:128])

# Erstelle ein Dropdown-Widget mit 'label' als Optionen und 'id' als tatsächlichen Wert
dropdown = widgets.Dropdown(
    options=dict(zip(df['label'], df.iloc[:, 0])),
    description='Zeitungstitel:',
)

# Erstelle einen Button mit der Bezeichnung 'Download'
button = widgets.Button(description="Download")

# Erstelle ein Output-Widget für die Anzeige von Ergebnissen oder Fortschritten
output = widgets.Output()

# Zeige das Dropdown, den Button und das Output-Widget an, angeordnet in einer horizontalen Box
display(widgets.HBox([dropdown, button]), output)

# Funktion, die aufgerufen wird, wenn der Button geklickt wird
def on_button_clicked(b):
    with output:
        # Deaktiviere den Button nach dem Klicken, um wiederholtes Klicken zu verhindern
        button.disabled = True
        
        # Rufe die Funktion 'download_newspaper_issues' mit dem ausgewählten Wert des Dropdowns auf
        download_newspaper_issues(dropdown.value, dropdown.value)

# Registriere die Funktion 'on_button_clicked' als Event-Handler für den Button-Click
button.on_click(on_button_clicked)

Output()